# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

city_data_df = pd.read_csv("../output_data/cities.csv")
# Display sample data
city_data_df = city_data_df.set_index('City_ID')
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,cidreira,-30.1811,-50.2056,69.19,97,93,15.93,BR,1699853915
1,mumbwa,-14.9783,27.0619,81.37,46,17,4.41,ZM,1699853916
2,port elizabeth,-33.9180,25.5701,62.89,82,40,14.97,ZA,1699853917
3,choyr,46.3611,108.3611,21.36,69,8,2.64,MN,1699853918
4,minas de marcona,-15.2119,-75.1103,57.15,93,90,13.31,PE,1699853918
5,colonia,50.9333,6.9500,48.74,91,100,5.75,DE,1699853722
6,kokopo,-4.3520,152.2633,81.03,75,100,11.68,PG,1699853920
7,bethel,41.3712,-73.4140,32.94,78,0,0.00,US,1699853921
8,megion,61.0296,76.1136,32.02,92,100,11.95,RU,1699853922


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
# Configure the map
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = .5,
    color = "City",
    hover_cols = ["City", "Humidity"]
)

# Display the map plot
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria
crit_temperature = (city_data_df['Max Temp'] < 70) & (city_data_df['Max Temp'] > 50)
crit_windSpeed = city_data_df['Wind Speed'] < 15
crit_humidity =  city_data_df['Humidity'] < 80
final_criteria = crit_temperature & crit_windSpeed & crit_humidity
# Drop any rows with null values
ideal_weather_df = city_data_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df = ideal_weather_df.set_index('City_ID')
# Display sample data
ideal_weather_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
9,blackmans bay,-43.0167,147.3167,69.93,40,4,5.01,AU,1699853922
30,ouargla,31.9493,5.3250,53.67,62,0,0.00,DZ,1699853938
59,gevas,38.2968,43.1083,53.20,49,39,4.32,TR,1699853962
63,hermanus,-34.4187,19.2345,62.69,72,29,13.27,ZA,1699853965
74,east london,-33.0153,27.9116,63.54,67,40,13.80,ZA,1699853974
92,sanctuary point,-35.1114,150.6290,69.98,58,57,7.99,AU,1699853987
95,tamanrasset,22.7850,5.5228,60.71,18,0,4.61,DZ,1699853989
101,newman,37.3138,-121.0208,58.05,58,97,1.01,US,1699853994
106,sabha,27.0377,14.4283,63.39,55,0,8.50,LY,1699853998


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
9,blackmans bay,AU,-43.0167,147.3167,40,
30,ouargla,DZ,31.9493,5.3250,62,
59,gevas,TR,38.2968,43.1083,49,
63,hermanus,ZA,-34.4187,19.2345,72,
74,east london,ZA,-33.0153,27.9116,67,
92,sanctuary point,AU,-35.1114,150.6290,58,
95,tamanrasset,DZ,22.7850,5.5228,18,
101,newman,US,37.3138,-121.0208,58,
106,sabha,LY,27.0377,14.4283,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "type": 'amenity',
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_lat = row['Lat']
    city_lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_lng},{city_lat},{radius}"
    params["bias"] = f"proximity:{city_lng},{city_lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
ouargla - nearest hotel: LYNATEL
gevas - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
east london - nearest hotel: No hotel found
sanctuary point - nearest hotel: Dungowan Accommodation
tamanrasset - nearest hotel: فندق الأمان
newman - nearest hotel: No hotel found
sabha - nearest hotel: فندق الكاظم-سبها
sonoita - nearest hotel: No hotel found
shahritus - nearest hotel: Marhabo Hotel
minas - nearest hotel: No hotel found
kashgar - nearest hotel: 深航国际酒店
kangding - nearest hotel: 溜溜城格桑花大酒店
buka - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
guerrero negro - nearest hotel: Plaza sal paraiso
forster - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
chorfa - nearest hotel: Hôtel Rabah Syphax
mahdia - nearest hotel: La Maison Bleue
laguna - nearest hotel: Holiday Inn Express & Suites
hasaki - nearest hotel: 大新旅館
bredasdorp - nearest hotel: Victoria Hotel
pletten

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
9,blackmans bay,AU,-43.0167,147.3167,40,Villa Howden
30,ouargla,DZ,31.9493,5.3250,62,LYNATEL
59,gevas,TR,38.2968,43.1083,49,No hotel found
63,hermanus,ZA,-34.4187,19.2345,72,Aloe guest house
74,east london,ZA,-33.0153,27.9116,67,No hotel found
...,...,...,...,...,...,...
494,illizi,DZ,26.4833,8.4667,48,بوناقة
531,rouissat,DZ,31.9243,5.3502,62,فندق الأروي الذهبي
535,proletarsk,RU,46.7031,41.7192,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)